# Использование RePlay
Данный ноутбук показывает базовый порядок работы с библиотекой.
- загрузка датасета
- деление на трейн и тест
- обучение модели
- сравнение с бейзлайном

Документацию можно собрать из папки `docs` командой `make html`. Она создаст папку `_build` с документацией -- `/docs/_build/html/index.html`

In [1]:
%load_ext autoreload
%autoreload 2

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark.
Для отладки моделей (не на кластере) создаём локальную сессию.

Объект `State` позволяет использовать одну и ту же сессию Spark в разных объектах.
Если сессия уже инициализирована, то её нужно передать при инициализации `State`. Модули библиотеки, которым необходимо использовать спарк-сессию, например для конвертации из пандас в спарк, будут искать сессию именно в `State`.

По умолчанию создастся дефолтная сессия. Простой способ получить сессию с заданным количеством выделенной памяти -- функция `get_spark_session` модуля `session_handler`.

In [2]:
from replay.session_handler import State

spark = State().session
spark

In [3]:
K = 10

## 0. Подготовка данных <a name='data-preparator'></a>
Популярные датасеты для рекомендательных систем можно найти в библиотеке rs_datasets.

Данные датасета доступны в виде `pandas.DataFrame` атрибутов объекта.
Посмотреть доступные данные можно с помощью метода `info`.

In [4]:
from rs_datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


### 0.1. DataPreparator

Внутренний формат данных в библиотеке - spark dataframe.
Модели replay ожидают на вход pandas или spark dataframe, содержащий обязательные столбцы item_id, user_id. Для некоторых моделей обязательно наличие столбца relevance, содержащего оценки релевантности объектов для пользователей.
Для разбиения данных по времени с помощью Splitter и фильтров в логе должен присутсововать столбец timestamp.

В replay есть DataPreparator, который автоматически конвертирует pandas и spark dataframe c логом в нужный формат (переименование/создание обязательных столбцов, корректное чтение дат, проверка отсутствия пропусков). DataPreparetor может конвертировать dataframe с признаками пользователей/объектов (нужно указать один из столбцов, соответсвующий user_id/item_id.

Также конвертировать pandas dataframes в spark можно с помощью вспомогательной функции convert_to_spark.

In [5]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)

In [6]:
log.show(3)

+-------+-------+---------+-------------------+
|user_id|item_id|relevance|          timestamp|
+-------+-------+---------+-------------------+
|      1|   1193|      5.0|2001-01-01 01:12:40|
|      1|    661|      3.0|2001-01-01 01:35:09|
|      1|    914|      3.0|2001-01-01 01:32:48|
+-------+-------+---------+-------------------+
only showing top 3 rows



In [7]:
from replay.utils import convert2spark
items = convert2spark(data.users)

### 0.2. Split

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе.

`UserSplitter` отбирает для теста некоторое количество или долю объектов для каждого пользователя.

In [8]:
from replay.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test = splitter.split(log)
(
    train.count(), 
    test.count()
)

(999709, 500)

## 1. Обучение модели 

#### SLIM
Один из простых, но эффективных алгоритмов 

In [9]:
from replay.models import SLIM

slim = SLIM(lambda_=0.01, beta=0.3)

/Users/a18785698/Documents/code_dir/base_replay_376/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [10]:
%%time

slim.fit(log=train)

01-Feb-21 15:00:00, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
01-Feb-21 15:00:00, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
01-Feb-21 15:00:04, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 3.52 s, sys: 210 ms, total: 3.73 s
Wall time: 14.3 s


In [11]:
%%time

recs = slim.predict(
    k=K,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

01-Feb-21 15:00:14, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM


CPU times: user 1.52 s, sys: 260 ms, total: 1.78 s
Wall time: 11.1 s


## 2. Оценка качества и сравнение результатов моделей

В библиотеке реализованы различные метрики качества рекомендательных систем, встречающихся в литературе.
Их можно использовать напрямую, либо запоминать результаты с помощью класса `Experiment`.

In [12]:
from replay.metrics import HitRate, NDCG, MAP
from replay.experiment import Experiment

metrics = Experiment(test, {NDCG(): K,
                            MAP() : K,
                            HitRate(): [1, int(K/2), K]})


In [13]:
%%time
metrics.add_result("SLIM", recs)
metrics.results

CPU times: user 150 ms, sys: 52.1 ms, total: 202 ms
Wall time: 1min 37s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.072,0.224,0.334,0.136348,0.18225


## 3. Конвертация в pandas

In [14]:
recs_pd = recs.toPandas()
recs_pd.head(3)

,user_id,item_id,relevance
0,1018,1200,0.819725
1,1018,1221,0.763315
2,1018,10,0.667240


## 4. Примеры использования других моделей RePlay

#### ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [17]:
from replay.models import ALSWrap

als = ALSWrap(rank=100)

In [18]:
%%time
als.fit(log=train)

01-Feb-21 15:05:47, replay, DEBUG: Начало обучения ALSWrap
DEBUG:replay:Начало обучения ALSWrap
01-Feb-21 15:05:47, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
01-Feb-21 15:05:48, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 315 ms, sys: 52.5 ms, total: 368 ms
Wall time: 1min 14s


In [19]:
%%time
recs = als.predict(
    k=K,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

01-Feb-21 15:07:01, replay, DEBUG: Начало предикта ALSWrap
DEBUG:replay:Начало предикта ALSWrap


CPU times: user 1.53 s, sys: 283 ms, total: 1.82 s
Wall time: 8.9 s


In [20]:
%%time
metrics.add_result("ALS", recs)
metrics.results

CPU times: user 173 ms, sys: 63.7 ms, total: 236 ms
Wall time: 22.7 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.072,0.224,0.334,0.136348,0.18225
ALS,0.064,0.258,0.376,0.149850,0.20300


#### MultVAE 
Пример использования DL в рекомендациях

In [21]:
from replay.models import MultVAE

multvae = MultVAE(epochs=500)

In [22]:
%%time
multvae.fit(log=train)

01-Feb-21 15:07:33, replay, DEBUG: Начало обучения MultVAE
DEBUG:replay:Начало обучения MultVAE
01-Feb-21 15:07:33, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
01-Feb-21 15:07:34, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
01-Feb-21 15:07:37, replay, DEBUG: Составление батча:
DEBUG:replay:Составление батча:
01-Feb-21 15:07:39, replay, DEBUG: Обучение модели
DEBUG:replay:Обучение модели
01-Feb-21 15:07:41, replay, DEBUG: Epoch[1] current loss: 1352.93728
DEBUG:replay:Epoch[1] current loss: 1352.93728
01-Feb-21 15:07:41, replay, DEBUG: Epoch[1] validation average loss: 1479.71289
DEBUG:replay:Epoch[1] validation average loss: 1479.71289
01-Feb-21 15:07:43, replay, DEBUG: Epoch[2] current loss: 1248.29758
DEBUG:replay:Epoch[2] current loss: 1248.29758
01-Feb-21 15:07:43, replay, DEBUG: Epoch[2] validation average loss: 1478.76685
DEBUG:replay:Epoch[2] validation average loss:

CPU times: user 20min 52s, sys: 2min 5s, total: 22min 58s
Wall time: 7min


In [23]:
%%time

recs = multvae.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

01-Feb-21 15:14:33, replay, DEBUG: Начало предикта MultVAE
DEBUG:replay:Начало предикта MultVAE
01-Feb-21 15:14:41, replay, DEBUG: Предсказание модели
DEBUG:replay:Предсказание модели
/Users/a18785698/Documents/code_dir/base_replay_376/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


CPU times: user 1.61 s, sys: 346 ms, total: 1.96 s
Wall time: 23.7 s


In [24]:
%%time
metrics.add_result("MultVAE", recs)
metrics.results

CPU times: user 168 ms, sys: 137 ms, total: 305 ms
Wall time: 48 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.072,0.224,0.334,0.136348,0.182250
ALS,0.064,0.258,0.376,0.149850,0.203000
MultVAE,0.012,0.056,0.132,0.036910,0.058415


## 5 Сравнение результатов различных моделей
С помощью experiment можно сравнить качество моделей, построенных с использованием различных инструментов.
Предположим, мы хотим сравнить модели RePlay с некой "внешней" моделью. Для этого нужно:
* 5.1. Экспортировать обучающую выборку (в pandas/numpy/csv)
* 5.2. Обучить модель и получить рекомендации для всех пользователей в виде csv/pandas-датафрейма
* 5.3. Считать рекомендации с помощью DataPreparator
* 5.4. Посчитать метрики в experiment

#### 5.1 Экспортируем train

In [25]:
train.toPandas().to_csv("train.csv", index=False)

In [26]:
!head -n 5 train.csv

user_id,item_id,relevance,timestamp
1,1029,5.0,2001-01-01 01:36:45
1,2294,4.0,2001-01-07 02:38:11
1,3114,4.0,2001-01-01 01:36:14
1,783,4.0,2001-01-07 02:38:11


#### 5.2 Обучаем модель и получаем рекомендации в формате `id пользователя - id объекта - relevance`

Предположим, что это произошло, и у нас есть рекомендации в виде csv-файла. Ниже в качестве пример используем рекомендации, полученные одной из моделей, с рандомными релевантностями.

In [27]:
from pyspark.sql.functions import rand

In [28]:
recs.withColumn('relevance', rand(seed=123)).toPandas().to_csv("recs.csv", index=False)

#### 5.3 Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [29]:
recs = DataPreparator().transform(
    path="recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

#### 5.4 Сравним качество внешней модели с предыдущими результатами

In [30]:
metrics.add_result("my_model", recs)
metrics.results.sort_values("NDCG@10", ascending=False)

,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
ALS,0.064,0.258,0.376,0.149850,0.203000
SLIM,0.072,0.224,0.334,0.136348,0.182250
MultVAE,0.012,0.056,0.132,0.036910,0.058415
my_model,0.014,0.046,0.132,0.035397,0.057047
